In [1]:
import pandas as pd
import os

Aggregate dataset

In [2]:
df = pd.read_csv('../data/cleaned/nopv_master.csv')
display(df)

,boro,block,lot,ease,address_1,address_2,address_3,"city,_state,_zip",country,tax_class,...,fiscal_year,date_published,published_year,published_month,published_day,rc1_desc,rc2_desc,rc3_desc,rc4_desc,rc5_desc
0,1,11,14,NaN,NaN,855 AVENUE OF THE AMERIC,NaN,NEW YORK NY 10001-4105,NaN,2,...,2025,2025-05-14,2025,5,14,Other Change,Unknown,Unknown,Unknown,Unknown
1,1,15,1004,NaN,NaN,1 S BOULEVARD,NaN,NYACK NY 10960-3604,NaN,4,...,2025,2025-05-14,2025,5,14,Fully Exempt and now restored to taxable,Unknown,Unknown,Unknown,Unknown
2,1,18,1134,NaN,NaN,88 GREENWICH ST,APT 718,NEW YORK NY 10006-2229,NaN,2,...,2025,2025-05-14,2025,5,14,Other Change,Unknown,Unknown,Unknown,Unknown
3,1,18,1160,NaN,NaN,88 GREENWICH ST,APT 815,NEW YORK NY 10006-2231,NaN,2,...,2025,2025-05-14,2025,5,14,Other Change,Unknown,Unknown,Unknown,Unknown
4,1,26,1008,NaN,NaN,55 EXCHANGE PL,NaN,NEW YORK NY 10005-3301,NaN,4,...,2025,2025-05-14,2025,5,14,Other Change,Unknown,Unknown,Unknown,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28193,5,8047,188,NaN,NaN,230 JOHNSON AVE,NaN,STATEN ISLAND NY 10307-1240,NaN,1,...,2025,2025-05-14,2025,5,14,Other Change,Unknown,Unknown,Unknown,Unknown
28194,5,8047,1001,NaN,NaN,2201 NEPTUNE AVE,NaN,BROOKLYN NY 11224-2375,NaN,4,...,2025,2025-05-14,2025,5,14,Alteration,Economics,Unknown,Unknown,Unknown
28195,5,8047,1002,NaN,NaN,2201 NEPTUNE AVE,NaN,BROOKLYN NY 11224-2375,NaN,4,...,2025,2025-05-14,2025,5,14,Alteration,Economics,Unknown,Unknown,Unknown
28196,5,8047,1003,NaN,NaN,2201 NEPTUNE AVE,NaN,BROOKLYN NY 11224-2375,NaN,2,...,2025,2025-05-14,2025,5,14,Alteration,Economics,Unknown,Unknown,Unknown


In [6]:
# for table class 1
df_c1 = df[(df['tax_class'] == '1' 
            #& df['date_published'] <= today # ! Update later
)]
display(df_c1)

,boro,block,lot,ease,address_1,address_2,address_3,"city,_state,_zip",country,tax_class,...,fiscal_year,date_published,published_year,published_month,published_day,rc1_desc,rc2_desc,rc3_desc,rc4_desc,rc5_desc
282,1,592,5,NaN,NaN,128 WEST WASHINGTON PLACE,NaN,NEW YORK NY 10014,NaN,1,...,2025,2025-05-14,2025,5,14,Other Change,Unknown,Unknown,Unknown,Unknown
283,1,595,82,NaN,NaN,488 GREENWICH STREET,NaN,NEW YORK NY 10013,NaN,1,...,2025,2025-05-14,2025,5,14,Other Change,Unknown,Unknown,Unknown,Unknown
314,1,626,29,NaN,NaN,48 HORATIO ST,NaN,NEW YORK NY 10014-1614,NaN,1,...,2025,2025-05-14,2025,5,14,Other Change,Unknown,Unknown,Unknown,Unknown
1119,1,1227,21,NaN,NaN,225 W 79TH ST,NaN,NEW YORK NY 10024-6205,NaN,1,...,2025,2025-05-14,2025,5,14,Other Change,Unknown,Unknown,Unknown,Unknown
1557,1,1322,20,NaN,NaN,1501 CHERRY ST,NaN,PHILADELPHIA PA 19102-1403,NaN,1,...,2025,2025-05-14,2025,5,14,Other Change,Unknown,Unknown,Unknown,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28190,5,8034,79,NaN,NaN,308 BETHEL AVE,NaN,STATEN ISLAND NY 10307-1360,NaN,1,...,2025,2025-05-14,2025,5,14,Other Change,Unknown,Unknown,Unknown,Unknown
28191,5,8035,7,NaN,NaN,7123 AMBOY RD,NaN,STATEN ISLAND NY 10307-1438,NaN,1,...,2025,2025-05-14,2025,5,14,Other Change,Unknown,Unknown,Unknown,Unknown
28192,5,8046,78,NaN,NaN,208 YETMAN AVE,NaN,STATEN ISLAND NY 10307-1202,NaN,1,...,2025,2025-05-14,2025,5,14,Other Change,Unknown,Unknown,Unknown,Unknown
28193,5,8047,188,NaN,NaN,230 JOHNSON AVE,NaN,STATEN ISLAND NY 10307-1240,NaN,1,...,2025,2025-05-14,2025,5,14,Other Change,Unknown,Unknown,Unknown,Unknown


In [9]:
df_c1_agg = df_c1.groupby('rc1_desc').size().reset_index(name = '# of Parcels')
display(df_c1_agg) # ! notice not all reasons are always non empty, need to make a list of all of them

,rc1_desc,# of Parcels
0,Alteration,16
1,Apportionment,4
2,Building in Progress Last Year,16
3,Demolition,128
4,Economics,616
5,Fully Exempt and now restored to taxable,2
6,New Building,12
7,New Building in Progress,30
8,Other Change,11444
9,Sales,12
